In [28]:
install.packages('class', repos='https://cran.r-project.org/')
library('class')

Installing package into 'C:/Users/User/Documents/R/win-library/3.3'
(as 'lib' is unspecified)


package 'class' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'class'"


The downloaded binary packages are in
	C:\Users\User\AppData\Local\Temp\RtmpAtUHWv\downloaded_packages


Add class package to the library for using knn function

In [21]:
creditDetails <- read.csv("german_credit_data1.csv")
head(creditDetails)

X.,Age,Sex,Job,Housing,Saving.accounts,Checking.account,Credit.amount,Duration,Purpose,Credit.Risks
0,67,male,2,own,NA,little,1169,6,radio/TV,1
1,22,female,2,own,little,moderate,5951,48,radio/TV,2
2,49,male,1,own,little,NA,2096,12,education,1
3,45,male,2,free,little,little,7882,42,furniture/equipment,1
4,53,male,2,free,little,little,4870,24,car,2
5,35,male,1,free,NA,NA,9055,36,education,1


Get data from the csv file

In [22]:
normalize <- function(x){ 
    (x-min(x))/(max(x)-min(x))
}

Declaring the normalize function to restrict the values in a column between 0-1

In [23]:
creditDetails <- as.data.frame(sapply(creditDetails, as.numeric),stringsAsFactors=FALSE)
creditDetails[is.na(creditDetails)] <- 0
normalCreditDetails <- as.data.frame(lapply(creditDetails,normalize))
head(creditDetails)

X.,Age,Sex,Job,Housing,Saving.accounts,Checking.account,Credit.amount,Duration,Purpose,Credit.Risks
0,67,2,2,2,0,1,1169,6,6,1
1,22,1,2,2,1,2,5951,48,6,2
2,49,2,1,2,1,0,2096,12,4,1
3,45,2,2,1,1,1,7882,42,5,1
4,53,2,2,1,1,1,4870,24,2,2
5,35,2,1,1,0,0,9055,36,4,1


Make data numeric and normalize all the columns

In [45]:
performKNN <- function(trainingSetPercent, creditDataFrame, predictionData){
    
    outputDataFrame <- data.frame(Samplings=character(), K=integer(), MisCalRate=double())
    newRow <- data.frame(Samplings=character(), K=integer(), MisCalRate=double())
	recordCount <- nrow(creditDataFrame)
    samplingRate <- trainingSetPercent*0.01
    testingLabelsCount <- recordCount* (1 - samplingRate)
    trainingDataIndices <- sample(1:recordCount, samplingRate*recordCount, replace=FALSE)
    trainingData <- subset(creditDataFrame[trainingDataIndices,], select=c("Age","Sex","Job","Housing","Saving.accounts","Checking.account","Credit.amount","Duration","Purpose"))
    testingDataIndices <- setdiff(1:recordCount,trainingDataIndices)
	testData <- subset(creditDataFrame[testingDataIndices,], select=c("Age","Sex","Job","Housing","Saving.accounts","Checking.account","Credit.amount","Duration","Purpose"))
	trainingLabels <- creditDataFrame$Credit.Risks[trainingDataIndices]
	testingTrueLabels <- creditDataFrame$Credit.Risks[testingDataIndices]
	kWithLowestMCRate <- 1
	LowestMCRate <- 1
	for(k in 1:20) {
		#print(k)
		predictedLabels <- knn(trainingData,testData,trainingLabels,k)
		numIncorrectLabels <- sum(predictedLabels != testingTrueLabels)
		misclassificationRate <- numIncorrectLabels/testingLabelsCount
		if(LowestMCRate > misclassificationRate){
			kWithLowestMCRate <- k
			LowestMCRate <- misclassificationRate
		}
		#print(misclassificationRate)
        newRow <- data.frame(Sampling=paste(toString(100-trainingSetPercent),toString(trainingSetPercent), sep="-"), K=k, MisCalRate=misclassificationRate)
        outputDataFrame <- rbind(outputDataFrame,newRow)
	}
	#print(kWithLowestMCRate)
	predictedLabel <- knn(trainingData, predictionData, trainingLabels, kWithLowestMCRate)
    print(predictedLabel)
	#print("Label --> ")
	return(outputDataFrame)
}

Created a function based on the code provided to perform knn for multiple inputs

In [46]:
predictionData <- data.frame(0.5678,1,0.6666667,0.5,0.5,0.3333333,0.43244057,0.64705882,0.4285714)
names(predictionData) <- c("Age","Sex","Job","Housing","Saving.accounts","Checking.account","Credit.amount","Duration","Purpose")
head(predictionData)

Age,Sex,Job,Housing,Saving.accounts,Checking.account,Credit.amount,Duration,Purpose
0.5678,1,0.6666667,0.5,0.5,0.3333333,0.4324406,0.6470588,0.4285714


Created dummy data for prediction based on the derived knn function.

In [47]:
misCalData <- data.frame(Samplings=character(), K=integer(), MisCalRate=double())
for(trainingSize in seq(from=50, to=90, by=10)){
    misCalData <- rbind(misCalData, performKNN(trainingSize,normalCreditDetails,predictionData))
}
misCalData

[1] 1
Levels: 0 1
[1] 1
Levels: 0 1
[1] 1
Levels: 0 1
[1] 1
Levels: 0 1
[1] 1
Levels: 0 1


Sampling,K,MisCalRate
50-50,1,0.3480
50-50,2,0.3320
50-50,3,0.3320
50-50,4,0.3220
50-50,5,0.3260
50-50,6,0.3260
50-50,7,0.3340
50-50,8,0.3220
50-50,9,0.3160
50-50,10,0.3160


The knn function has been applied for multiple sample data and for each sample the k value is varied from 1-20

References:
http://www.cse.buffalo.edu/~bina/cse487/spring2017/Lectures/classnotes.html#Part1